# Ansible role

## 1. Getting started

### 1.1. Role directory structure

```
roles
 ├╌ <role name>
 │     ├╌ tasks
 │     ├╌ handlers
 │     ├╌ library
 │     ├╌ defaults
 │     ├╌ vars
 │     ├╌ files
 │     ├╌ templates
 │     └╌ meta
 ├╌ ansible.cfg
 ├╌ ssh_config
 ├╌ <vault-id>
 ├╌ inventory
 └╌ <playbook>.yml
```

- `tasks/main.yml`: The main list of tasks that the role executes
- `handlers/main.yml`: Handlers, which may be used within or outside this role
- `library/my_module.py`: Modules, which may be used within this role
- `defaults/main.yml`: Default variables for the role. These variables have the lowest priority of any variables available, and can be easily overridden by any other variable, including inventory variables
- `vars/main.yml`: Other variables for the role
- `files/main.yml`: Files that the role deploys
- `templates/main.yml`: Templates that the role deploys
- `meta/main.yml`: Metadata for the role, including role dependencies

### 1.2. Storing and finding roles

By default, Ansible looks for roles in two locations:
- in a directory called `roles/`, relative to the playbook file
- in `/etc/ansible/roles`

Alternatively, you can call a role with a fully qualified path:
```yml
---
- hosts: webservers
  roles:
    - role: '/path/to/my/roles/common'
```

### 1.3. Using roles

You can use roles in three ways:
- At the play level with the `roles` option: This is the classic way of using roles in a play.
- At the tasks level with `include_role`: You can reuse roles dynamically anywhere in the `tasks` section of a play using `include_role`.
- At the tasks level with `import_role`: You can reuse roles statically anywhere in the `tasks` section of a play using `import_role`.

#### 1.3.1. Using roles at the play level

The classic (original) way to use roles is with the `roles` option for a given play:
```yml
---
- hosts: webservers
  roles:
    - common
    - webservers
```

When you use the `roles` option at the play level, for each role `x`:
- If `roles/x/tasks/main.yml` exists, Ansible adds the tasks in that file to the play.
- If `roles/x/handlers/main.yml` exists, Ansible adds the handlers in that file to the play.
- If `roles/x/vars/main.yml` exists, Ansible adds the variables in that file to the play.
- If `roles/x/defaults/main.yml` exists, Ansible adds the variables in that file to the play.
- If `roles/x/meta/main.yml` exists, Ansible adds any role dependencies in that file to the list of roles.
- Any 'copy', 'script', 'template' or 'include tasks' (in the role) can reference files in `roles/x/{files,templates,tasks}/` (dir depends on task) without having to path them relatively or absolutely.

When you use the `roles` option at the play level, Ansible treats the roles as static imports and processes them during playbook parsing. Ansible executes your playbook in this order:
- Any `pre_tasks` defined in the play.
- Any `handlers` triggered by `pre_tasks`.
- Each role listed in `roles:`, in the order listed. Any role dependencies defined in the role’s `meta/main.yml` run first, subject to tag filtering and conditionals.
- Any `tasks` defined in the play.
- Any `handlers` triggered by the `roles` or `tasks`.
- Any `post_tasks` defined in the play.
- Any `handlers` triggered by `post_tasks`.

> If using tags with tasks in a role, be sure to also tag your `pre_tasks`, `post_tasks`, and `role` dependencies and pass those along as well, especially if the pre/post tasks and role dependencies are used for monitoring outage window control or load balancing. See Tags for details on adding and using tags.

You can pass other keywords to the `roles` option:
```yml
---
- hosts: webservers
  roles:
    - common
    - role: foo_app_instance
      vars:
        dir: '/opt/a'
        app_port: 5000
      tags: typeA
    - role: foo_app_instance
      vars:
        dir: '/opt/b'
        app_port: 5001
      tags: typeB
```

When you add a tag to the `role` option, Ansible applies the tag to ALL tasks within the role.

When using `vars:` within the `roles:` section of a playbook, the variables are added to the play variables, making them available to all tasks within the play before and after the role. This behavior can be changed by `DEFAULT_PRIVATE_ROLE_VARS`

In [ ]:
ansible-playbook role.yml --tags "G1"